In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# 1. Cargar tus datos preprocesados
# Se asume que el CSV tiene al menos dos columnas: 'text' y 'label'
df = pd.read_csv("processedstopword1.csv")

# Convertir la etiqueta a formato numérico si es necesario (por ejemplo, 0: noticia real, 1: noticia falsa)
# Asegúrate de que la columna 'label' esté en el formato adecuado para la clasificación

# 2. Dividir los datos en entrenamiento y prueba
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 3. Convertir a datasets de Hugging Face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 4. Seleccionar el modelo y cargar el tokenizador
# Usa "bert-base-spanish-wwm-cased" si tus noticias están en español, o "bert-base-uncased" para inglés.
model_name = "bert-base-spanish-wwm-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# 5. Tokenizar los textos
def tokenize_data(example):
    return tokenizer(example['text'], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(tokenize_data, batched=False)
test_dataset = test_dataset.map(tokenize_data, batched=False)

# 6. Configurar el modelo para clasificación (dos clases: noticia real o falsa)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 7. Definir argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,                      # Número de épocas (ajusta según tus datos)
    per_device_train_batch_size=8,           # Tamaño de batch para entrenamiento
    per_device_eval_batch_size=8,            # Tamaño de batch para evaluación
    evaluation_strategy="epoch",             # Evaluación al final de cada época
    save_strategy="epoch",                   # Guarda el modelo al final de cada época
    learning_rate=2e-5,
    weight_decay=0.01,
)

# 8. Función para calcular métricas (por ejemplo, exactitud)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = (predictions == torch.tensor(labels)).float().mean()
    return {"accuracy": accuracy.item()}

# 9. Configurar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# 10. Entrenar el modelo
trainer.train()
